# 로지스틱 회귀 (Logistic Regression )
`04_logistic_regression.ipynb`

In [ ]:
import pandas as pd
import numpy as np
import warnings

warnings.filterwarnings('ignore')

In [ ]:
fish_df = pd.read_csv('./fish_data.csv')
fish_df.head()

In [ ]:
# 데이터셋 분리
X = fish_df[['Weight', 'Length', 'Diagonal', 'Height', 'Width']].to_numpy()
y = fish_df['Species'].to_numpy()

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
len(X_train)


In [ ]:
# 스케일링 (훈련 셋에서 사용한 기준으로 테스트셋을 스케일링 해야함!)
from sklearn.preprocessing import StandardScaler

ss = StandardScaler()
ss.fit(X_train)
X_train = ss.transform(X_train)
X_test = ss.transform(X_test)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

# 주변 3개의 생선을 보고 확률을 계산
kn = KNeighborsClassifier(n_neighbors=3)
kn.fit(X_train, y_train)

kn.score(X_train, y_train), kn.score(X_test, y_test)

In [ ]:
# KN분류기가 분류해내는 클래스 목록
print(kn.classes_)
# 테스트셋에서 앞 5개 데이터를 예측해라
print(kn.predict(X_test[:5]))

In [ ]:
# 테스트 데이터 5개가 특정 생선일 확률
kn.predict_proba(X_test[:5])

## 로지스틱 회귀
확률을 구할 때 시그모이드 함수를 사용해서 0~1로 변환

### 로지스틱 회귀로 이진분류 (빙어, 도미)

In [ ]:
import matplotlib.pyplot as plt

# 시그모이드 함수(모든 x를 0 ~ 1 사이 값으로 변환)
z = np.arange(-5, 5, 0.1)
phi = 1 / (1 + np.exp(-z))
plt.plot(z, phi)
plt.show()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

ss = StandardScaler()
ss.fit(X_train)
X_train_s = ss.transform(X_train)
X_test_s = ss.transform(X_test)


index = (y_train == 'Bream') | (y_train == 'Smelt')

X_bream_smelt_train = X_train_s[index]
y_bream_smelt_train = y_train[index]

In [ ]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
# 로지스틱회귀 학습 -> 방정식 만들기 -> 두개중 하나로 분류해야 함 -> abc순 뒤에걸로 분류할 점수를 계산하는 식
lr.fit(X_bream_smelt_train, y_bream_smelt_train)
print(f'=== 방정식 기울기, 절편 결과 : {lr.coef_} / {lr.intercept_}')
# 분류하는 클래스 명
print(f'\n 분류해낼 클래스 두개 (abc순) -> {lr.classes_}')

# 도미/빙어 예측 결과
print(lr.predict(X_bream_smelt_train[:5]))
print(lr.score(X_bream_smelt_train, y_bream_smelt_train))

# 실제 z값
descions = lr.decision_function(X_bream_smelt_train[:5])
print(f'\n=== 실제z값 ===')
print(descions)

# 직접 시그모이드 통과시키기
from scipy.special import expit
print('\n=== 시그모이드 함수 통과 후 값 (직접확인) ===')
print(expit(descions))

# 바로 계산된 결과
print('\n=== lr 이 바로 계산해준 값 ===')
print(lr.predict_proba(X_bream_smelt_train[:5]))



### 로지스틱 회귀로 다중분류 (생선 7종)

In [ ]:
# 2진분류 vs 다중분류 에 따라 내부 동작이 다름

# C=규제, max_iter=반복횟수의 제한
lr = LogisticRegression(C=20, max_iter=1000)

lr.fit(X_train_s, y_train)
# 훈련, 테스트 점수
print(lr.score(X_train_s, y_train))
print(lr.score(X_test_s, y_test))

In [97]:
print(lr.predict(X_test_s[:3]))

# 분류의 근거 (확률)
print(lr.classes_)
print(np.round(lr.predict_proba(X_test_s[:3]), decimals=3))

['Perch' 'Smelt' 'Pike']
['Bream' 'Parkki' 'Perch' 'Pike' 'Roach' 'Smelt' 'Whitefish']
[[0.    0.014 0.842 0.    0.135 0.007 0.003]
 [0.    0.003 0.044 0.    0.007 0.946 0.   ]
 [0.    0.    0.034 0.934 0.015 0.016 0.   ]]


In [ ]:
# 각 생선별 z값 구하는 함수의 계수 확인 (생선 7개 -> 함수 7개)
lr.coef_

In [ ]:
# Z 값으로 보기
decisions = lr.decision_function(X_test_s[:3])
print(np.round(decisions, decimals=2))

# 소프트맥스 함수에 넣어서 확률로 바꾸기
from scipy.special import softmax
proba = softmax(decisions, axis=1)
print(np.round(proba, decimals=3))

[[ -6.51   1.04   5.17  -2.76   3.34   0.35  -0.63]
 [-10.88   1.94   4.78  -2.42   2.99   7.84  -4.25]
 [ -4.34  -6.24   3.17   6.48   2.36   2.43  -3.87]]
[[0.    0.014 0.842 0.    0.135 0.007 0.003]
 [0.    0.003 0.044 0.    0.007 0.946 0.   ]
 [0.    0.    0.034 0.934 0.015 0.016 0.   ]]


: 